In [1]:
# Import necessary libraries
import cv2

In [2]:
# Initialize video capture
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Initialize the initial frame for comparison
initial_frame = None

# Set the frame skip rate (For debugging purposes)
normal_frame_skip_rate = 1
fast_forward_skip_rate = 10
frame_skip_rate = normal_frame_skip_rate

# Create an infinite loop to keep reading frames
while True:
    check, frame = video.read()
    if not check:
        break

    # If frame_skip_rate more than 1, skip frames based on frame_skip_rate
    if frame_skip_rate > 1:
        for _ in range(frame_skip_rate - 1):
            video.grab()  # Skip frames

    # Convert the frame to grayscale and use gaussian blur to reduce noise
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

    # Save the first frame as the initial frame
    if initial_frame is None:
        initial_frame = blur_frame

    # Compute the difference between the initial frame and the current frame
    delta_frame = cv2.absdiff(initial_frame, blur_frame)

    # Convert the difference (delta_frame) to a binary image.
    # If the pixel value is greater than 30, it will white, else it will be black.
    threshold, threshold_frame = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)

    # Apply morphological closing to close small gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))
    closed_frame = cv2.morphologyEx(threshold_frame, cv2.MORPH_CLOSE, kernel)

    # Find contours in the thresholded frame
    contours, threshold = cv2.findContours(closed_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through each detected contour
    for contour in contours:
        if cv2.contourArea(contour) < 1500: # Ignore small contours
            continue

        # Draw a green rectangle around the detected contour
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the frame with the bounding boxes
    cv2.imshow("Detected Objects", frame)

    # Check for key press events
    key = cv2.waitKey(30) & 0xFF  # Capture the pressed key
    if key == 27:  # Exit when "Esc" key is pressed
        break
    elif key == ord('d'):
        frame_skip_rate = fast_forward_skip_rate
    elif key == ord('a'):
        frame_skip_rate = normal_frame_skip_rate

# Release resources and close the window
video.release()
cv2.destroyAllWindows()